In [ ]:
import requests
from dataclasses import dataclass

@dataclass
class GenOpenApi:
    type: dict[str, str]
    api: dict[str, str]

"""
{
    tags: {
        'type': {
            type_name: 'type'
        }
        api: {
            api_name: 'api'
        }
    } 
}
"""

temp_schemas = """
type %s = {
%s}
"""

temp_api = """
export async function %s {
  const url = %s;
  const options = {
    method: "%s",
    headers: getHeaders(),
    %s
  };
  return await %s(url, options);
}
"""

class OpenApiHepler:
    def __init__(self, host: str = "127.0.0.1", port: str = "8000", path: str = "/api/openapi.json", url: str = None,):
        self.host: str = host,
        self.port: str = port,
        self.path= path
        self.url= url if url else f"http://{host}:{port}{path}"
        
        # openapi json
        # base
        self.openapi_json: dict[str, dict] = self.get_openapi_json()
        self.components: dict[str, dict] = self.get_components()
        self.schemas: dict[str, dict[dict]] = self.get_schemas()
        
        # info list
        self.paths: dict[str, dict] = self.get_paths()
        self.tags: list[str] = self.get_tags()
        
        # main
        self.types: dict[str, list] = self.get_all_types()
        self.apis: dict[str, list] = self.get_all_apis()
        #
        self.client : dict[str, GenOpenApi] = self.gen_client()
        
    def get_openapi_json(self):
        try:
            response = requests.get(self.url)
            if response.status_code == 200:
                res = response.json()
                return res
            else:
                print(
                    "Failed to retrieve data. Status code:",
                    response.status_code,
                )
        except requests.RequestException as e:
            print("Error during request:\n", e)
    
    def get_components(self) -> dict[str, dict]:
        if 'components' in self.openapi_json:
            return self.openapi_json['components']
        
    def get_schemas(self) -> dict[str, dict]:
        if 'schemas' in self.components:
            return  self.components['schemas']
    
    def get_paths(self) -> dict[str, dict]:
        if 'paths' in self.openapi_json:
            return self.openapi_json['paths']
    
    def get_tags(self) -> list[str]:
        tags: set[str] = set()
        paths: dict[str, dict] = self.paths
        if paths:
            for path in paths:
                for method in self.paths[path]:
                    if 'tags' in self.paths[path][method]:
                        for tag in self.paths[path][method]['tags']:
                            tags.add(tag)
        return list(tags)
    
    def get_path_names(self) -> list[str]:
        path_names: list[str] = []
        paths: dict[str, dict] = self.paths
        if paths:
            for path in paths:
                path_names.append(path)
        return path_names
    
    def get_all_apis(self):
        apis_dict = {}
        if self.openapi_json and 'paths' in self.openapi_json:
            for api_path, api_path_item in self.openapi_json['paths'].items():
                function_name = ''
                for method, api_operation in api_path_item.items():
                    function_name = api_operation['summary'].replace(' ', '')
                    responses = api_operation["responses"]
                    
                    if api_operation.get("requestBody"):
                        api_schemas = api_operation["requestBody"]["content"]
                        if api_schemas.get("application/json"):
                            api_schemas = api_schemas["application/json"]["schema"]["$ref"].split("/")[-1]
                            api_body = "\n\t\tbody: JSON.stringify(data)" if len(api_schemas) != 0 else ""
                        elif api_schemas.get("multipart/form-data"):
                            api_schemas = api_schemas["multipart/form-data"]["schema"]["properties"]
                            api_schemas = "FormData"
                            api_body = "\n\t\tbody: data" if len(api_schemas) != 0 else ""
                        api_parameters = (" ," if len(api_parameters) else "") + api_parameters
                        api_name += f"(data: {api_schemas} {api_parameters})"
                    else:
                        api_name += f"({api_parameters})"
                    
                    if responses["200"].get("content"):
                        schema = responses["200"]["content"]["application/json"]["schema"]
                        api_responses = schema["$ref"].split("/")[-1] if schema.get("$ref") else ""
                        function_name += f": Promise<{api_responses}>" if len(api_responses) > 0 else ""

                    api_body = ''
                    api_text = temp_api % (function_name, api_path, method.upper(), api_body, "api" if "download" not in function_name else "apiFile",)
                apis_dict[api_path] = api_text
        return apis_dict
    
    def get_all_types(self):
        types_dict = {}
        if self.schemas:
            for schemas_name, schemas_item in self.schemas.items():
                type_text = ""
                for item_name, item_type in schemas_item["properties"].items():
                    required = '' if item_name in schemas_item['required'] else '?'
                    if item_type.get("anyOf"):
                        anyOf = " | ".join([anyType["type"] for anyType in item_type.get("anyOf")])
                        type_text += f"  {item_name}{required}: {anyOf}\n"
                    else:
                        item_type = item_type.get("type")
                        type_text += f"  {item_name}{required}: {item_type}\n"
                
                type_str = temp_schemas % (schemas_name, type_text)
                type_str = type_str.replace("array", "[]").replace("integer", "number")
                
                types_dict[schemas_name] = type_str
            return types_dict
        
    def gen_client(self):
        pass
            
from pprint import pprint
openapi_hepler = OpenApiHepler()
pprint(openapi_hepler.apis)

{'/api/group': '\n'
               'export async function CreateGroup {\n'
               '  const url = /api/group;\n'
               '  const options = {\n'
               '    method: "POST",\n'
               '    headers: getHeaders(),\n'
               '    \n'
               '  };\n'
               '  return await api(url, options);\n'
               '}\n',
 '/api/group/{uuid}': '\n'
                      'export async function DeleteGroup {\n'
                      '  const url = /api/group/{uuid};\n'
                      '  const options = {\n'
                      '    method: "DELETE",\n'
                      '    headers: getHeaders(),\n'
                      '    \n'
                      '  };\n'
                      '  return await api(url, options);\n'
                      '}\n',
 '/api/todo': '\n'
              'export async function GetTodos: Promise<ToDoList> {\n'
              '  const url = /api/todo;\n'
              '  const options = {\n'
              '  